# Stats

## fold = 5 (early stopping=50)
precision: 0.647
recall: 0.931
f1: 0.763

## fold = 5 (early stopping=100)
precision: 0.666
recall: 0.929
f1: 0.776

## fold = 5 (early_stopping=10)
precision: 0.618
recall: 0.934
f1: 0.744

## fold = 5 (early_stopping=25)
precision: 0.642
recall: 0.933
f1: 0.761

**early_stopping 값이 커질수록 precision이 높아짐. recall은 거의 그대로**

## fold = 10 (early_stopping=5)
public score: 0.7390581717451523
precision: 0.603
recall: 0.94
f1: 0.734

- iteration: 300
precision: 0.614
recall: 0.939
f1: 0.742

## fold = 10 (early_stopping=10)
public score: 0.7330677290836655
precision: 0.633
recall: 0.94
f1: 0.756

## fold = 10 (early_stopping=25)
public score: 0.7289611143354615
precision: 0.65
recall: 0.94
f1: 0.769

## fold = 10 (early stopping=50)
public score: 0.7189542483660131
precision: 0.665
recall: 0.938
f1: 0.778

## fold = 11 (early_stopping=5)
public score: 0.7404921700223713
precision: 0.606
recall: 0.943
f1: 0.738

## fold = 12 (early_stopping=5)
public score: 0.7271714922048997
precision: 0.6
recall: 0.943
f1: 0.733

## fold = 13 (early_stopping=5)
precision: 0.609
recall: 0.942
f1: 0.74

## fold = 15 (early_stopping=10)
precision: 0.634
recall: 0.94
f1: 0.757

## fold = 15 (early_stopping=5)
public score: 0.7369601794727987
precision: 0.608
recall: 0.939
f1: 0.738

# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [1]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    make_scorer,
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    StratifiedKFold
)
from sklearn.base import clone

from catboost import CatBoostClassifier

### 데이터 셋 읽어오기

In [3]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [4]:
df_train.head() # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True


## 2. 데이터 전처리

In [5]:
# bant_submit 전처리
df_train['bant_submit'] = df_train['bant_submit'].apply(
    lambda x: 0 if x == 0 else (1 if x == 0.25 else (2 if x == 0.5 else (3 if x == 0.75 else 4)))
)
df_test['bant_submit'] = df_test['bant_submit'].apply(
    lambda x: 0 if x == 0 else (1 if x == 0.25 else (2 if x == 0.5 else (3 if x == 0.75 else 4)))
)

In [6]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

In [7]:
df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

In [8]:
# customer_job 전처리
df_all['customer_job'].replace(to_replace=['systems engineer','system engineer'], value="engineering", inplace=True)
df_all['customer_job'].replace(to_replace=['information technology', 'information_technology','it - information technology',  'it', 'information technology\u200b',], value="IT", inplace=True)
df_all['customer_job'].replace(to_replace=['human resources','human_resources','hr',], value="administration", inplace=True)
df_all['customer_job'].replace(to_replace=['director it', 'it director','director,it','director of it',"i'm directing it",'it dairector',], value="IT director", inplace=True)
df_all['customer_job'].replace(to_replace=['research and developement','research & development'], value="research and development", inplace=True)
df_all['customer_job'].replace(to_replace=['other', 'nothing','otro','others', 'otros', 'n.a','sonstiges', 'altro','egyéb','autres'], value="other", inplace=True)
df_all['customer_job'].replace(to_replace=['sales', 'sale','vendite'], value="sales", inplace=True)
df_all['customer_job'].replace(to_replace=['media and communication','media and communications', 'media_e_comunicazione','média_és_kommunikáció','medien_und_kommunikation','medios_de_comunicación','media_and_communication'],value='media and communication', inplace=True)
df_all['customer_job'].replace(to_replace=['healthcare services', 'healthcare_services'], value='healthcare services', inplace=True)
df_all['customer_job'].replace(to_replace=['pénzügy','finanzen','finanzas'], value='finance', inplace=True)
df_all['customer_job'].replace(to_replace=['arts and design','arts_and_design','art and design','művészet_és_design', 'arte_e_design','kreation und design','kreation_und_design','arte y diseño'], value='art and design', inplace=True)
df_all['customer_job'].replace(to_replace=['military and protective services','military_and_protective_services'], value='military', inplace=True)

In [9]:
# product category 전처리
df_all['product_category'].replace(to_replace=['etc.','outros', 'other','autre', 'khác','آخر','not specified',], value="other", inplace=True)
df_all['product_category'].replace(to_replace=['חימום','aquecimento',], value="heating", inplace=True)
df_all['product_category'].replace(to_replace=['one:quick series','lg one:quick','lg one:quick series', 'one quick works''onequick series','one quick:flex','one:quick flex','one:quick'], value="one quick", inplace=True)
df_all['product_category'].replace(to_replace=['מזגנים למקום מגורים','ar condicionado residencial','เครื่องปรับอากาศเผื่อที่อยู่อาศัย', 'aire acondicionado residencial','climatiseur résidentiel'], value="residential air conditioner", inplace=True)
df_all['product_category'].replace(to_replace=['酒店電視'], value="hotel tv", inplace=True)
df_all['product_category'].replace(to_replace=['軟體'], value="software", inplace=True)
df_all['product_category'].replace(to_replace=['multi-split (plusieurs pièces)'], value="multi-split", inplace=True)

In [10]:
# customer_type 전처리
df_all['customer_type'].replace(to_replace=['End-Customer', 'End Customer',], value="end customer", inplace=True)
df_all['customer_type'].replace(to_replace=['Other','Others'], value="other", inplace=True)
df_all['customer_type'].replace(to_replace=['Software/Solution Provider', 'Software / Solution Provider'], value="software/solution provider", inplace=True)

In [11]:
special_characters = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']

def remove_special_characters(text):
    for char in special_characters:
        text = text.replace(char, '')
    return text

for col in label_columns:
    df_all[col] = df_all[col].apply(
        lambda x: x if pd.isna(x) else remove_special_characters(x.replace(' ', '')).upper()
    )

In [12]:
# inquiry_type 전처리
df_all['inquiry_type'] = df_all['inquiry_type'].replace('OTHER', 'ETC').replace('OTHERS', 'ETC')

In [13]:
# expected_timeline 전처리

def preprocess_timeline(timeline):
    # 빈도수가 너무 높아 변형하지 않을 값
    allowed_fixed = [
        'LESSTHAN3MONTHS', '3MONTHS6MONTHS', '6MONTHS9MONTHS', '9MONTHS1YEAR', 'MORETHANAYEAR',
        'LESSTHAN6MONTHS'
    ]
    allowed_date = [
        # 기간이 올바르게 정해진 값
        'DAY', 'MONTH', 'YEAR',
        'JAN', 'FEB', 'MAR', 'APRIL', 'MAY', 'JUNE', 'JULY', 'AUGUST', 'SEPTEMBER', 'OCTOBER', 'NOVEMBER', 'DECEMBER'
    ]
    # BEINGFOLLOWEDUP으로 통합할 값
    allowed_fu = [
        'FOLLOWEDUP', 'FOLLOWINGUP', 'FUUNDERPROGRESS', 'FOLLOWUP'
    ]
    # DROPPED로 통합할 값
    allowed_dropped = [
        'DROP', 'CLOSING', 'PROBLEM', 'ISSUE'
    ]
    # NOTCONNECTED로 통합할 값
    allowed_connect = [
        'SWITCHEDOFF', 'NORESPON', 'NOTANSWER', 'NOTCONNECT', 'NOTREACH', 'NOTRESPON',
        'NTRESPON', 'NTANSWER', 'NTCON', 'NTREACH'
    ]
    # NOREQ로 통합할 값
    allowed_no_req = [
        # 기한을 요구하지 않은 값
        'NOTHAVINGANYREQUIRE', 'NOTINTEREST', 'NOREQ', 'NOINTEREST',
        'NOTREQ'
    ]
    # DISCUSSWITHCLI로 통합할 값
    allowed_discussed = [
        'DISCUSSEDWITHCLI', 'DISCUSSEDWITHTHE'
    ]
    for word in allowed_fixed:
        if word in timeline:
            return timeline
    for word in allowed_date:
        if word in timeline:
            return 'DATESPECIFIED'
    for word in allowed_fu:
        if word in timeline:
            return 'FOLLOWEDUP'
    for word in allowed_dropped:
        if word in timeline:
            return 'DROPPED'
    for word in allowed_connect:
        if word in timeline:
            return 'NOTCONNECTED'
    for word in allowed_no_req:
        if word in timeline:
            return 'NOREQ'
    for word in allowed_discussed:
        if word in timeline:
            return 'DISCUSSEDWITHCLI'
    return timeline

df_all['expected_timeline'] = df_all['expected_timeline'].apply(
    lambda x: x if pd.isna(x) else preprocess_timeline(x)
)

In [14]:
# business_subarea 전처리
df_all['business_subarea'] = df_all['business_subarea'].replace(
    'OTHERSTORES', 'OTHERS').replace(
    'MANUFACTURINGFACTORYPLANT', 'MANUFACTURING').replace(
    'GENERALHOSPITAL', 'HOSPITAL').replace(
    'PHARMACEUTICAL', 'PHARMACY').replace(
    'ELECTRONICSTELCO', 'TELECOMMUNICATION')

### 레이블 인코딩

In [15]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [16]:
# label encoding
for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

다시 학습 데이터와 제출 데이터를 분리합니다.

In [17]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

### 2-2. 학습, 검증 데이터 분리

In [18]:
used_features = [
    'bant_submit', 
    'customer_country', 
    'business_unit',
    'com_reg_ver_win_rate', 
    'customer_idx', 
    'customer_type', 
    'enterprise',
    'historical_existing_cnt', 
    'id_strategic_ver', 
    'it_strategic_ver',
    'idit_strategic_ver', 
    'customer_job', 
    'lead_desc_length',
    'inquiry_type', 
    'product_category', 
    'product_subcategory',
    'product_modelname', 
    'customer_country.1', 
    'customer_position',
    'response_corporate', 
    'expected_timeline', 
    'ver_cus', 
    'ver_pro',
    'ver_win_rate_x', 
    'ver_win_ratio_per_bu', 
    'business_area',
    'business_subarea', 
    'lead_owner', 
    # 'is_converted'
]

## 3. 모델 학습

### 모델 학습

In [19]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [20]:
n_folds = 12  # fold 개수
skfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# 여러 model의 predict_proba 결과가 저장되는 곳
proba_pred = np.zeros((df_test.shape[0], 2))

precision_scores = []
recall_scores = []
f1_scores = []

for i, (train_idx, val_idx) in enumerate(skfold.split(df_train[used_features], df_train['is_converted'])):
    X_train, X_val = df_train[used_features].iloc[train_idx], df_train[used_features].iloc[val_idx]
    y_train, y_val = df_train['is_converted'].iloc[train_idx].astype(int), df_train['is_converted'].iloc[val_idx].astype(int)
    X_test = df_test[used_features]
    
    neg_pos_ratio = np.sum(y_train == 0) / np.sum(y_train == 1)
    
    model = CatBoostClassifier(
        scale_pos_weight=neg_pos_ratio,
        random_seed=42,
        early_stopping_rounds=5,
        per_float_feature_quantization=[
            '4:border_count=1024'
        ],
        cat_features=[
            used_features.index('bant_submit'),
            used_features.index('customer_country'),
            used_features.index('business_unit'),
            used_features.index('customer_type'),
            used_features.index('enterprise'),
            used_features.index('customer_job'),
            used_features.index('inquiry_type'),
            used_features.index('product_category'),
            used_features.index('product_subcategory'),
            used_features.index('product_modelname'),
            used_features.index('customer_country.1'),
            used_features.index('customer_position'),
            used_features.index('response_corporate'),
            used_features.index('expected_timeline'),
            used_features.index('business_area'),
            used_features.index('business_subarea'),
        ],  # train set에서 categorical feature의 indices
        verbose=False
    )
    
    model.fit(
        X_train.fillna(-1), y_train,
        eval_set=(X_val.fillna(-1), y_val)
    )
    y_pred = model.predict(X_val.fillna(-1))
    print(f"************** {i}-th **************")
    confusion = confusion_matrix(y_val, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred, labels=[True, False])
    recall = recall_score(y_val, y_pred)
    F1 = f1_score(y_val, y_pred, labels=[True, False])
    print(confusion)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(F1)
    print(f"Best Iteration: {model.best_iteration_}, Best Score: {model.best_score_['validation']['Logloss']}")
    print("************************************")
    
    test_pred = model.predict_proba(X_test.fillna(-1))
    proba_pred += (test_pred / n_folds)

print(f"precision: {np.array(precision_scores).mean().round(3)}")
print(f"recall: {np.array(recall_scores).mean().round(3)}")
print(f"f1: {np.array(f1_scores).mean().round(3)}")

************** 0-th **************
[[ 387   18]
 [ 260 4277]]
Best Iteration: 267, Best Score: 0.1405927131921905
************************************
************** 1-th **************
[[ 389   16]
 [ 249 4288]]
Best Iteration: 255, Best Score: 0.13462175457418207
************************************
************** 2-th **************
[[ 379   25]
 [ 274 4264]]
Best Iteration: 204, Best Score: 0.15577605282582796
************************************
************** 3-th **************
[[ 384   20]
 [ 226 4312]]
Best Iteration: 278, Best Score: 0.13214344269169595
************************************
************** 4-th **************
[[ 382   22]
 [ 241 4297]]
Best Iteration: 276, Best Score: 0.1371760046640828
************************************
************** 5-th **************
[[ 372   32]
 [ 285 4253]]
Best Iteration: 173, Best Score: 0.17477853941240612
************************************
************** 6-th **************
[[ 390   14]
 [ 282 4256]]
Best Iteration: 231, Best Sc

## 4. 제출하기

### 테스트 데이터 예측

In [21]:
test_pred = np.argmax(proba_pred, axis=1)
sum(test_pred)

1869

### 제출 파일 작성

In [22]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**